# Generating Rhyming Couplets using The Dartmouth
This project explores a few different ways that we can create rhyming couplets using sentences from articles appearing daily in The Dartmouth. The first sentences of the couplets are real sentences that appeared in articles that day (or the preceding few days). The second sentences are generated by the computer using different techniques including POS tagging and bigram dictionaries. Doing it this way does not always produce perfect couplets, but keeping the first lines real often preserves a sense of the news. Also by offering up a few different ways to generate couplets, it offers viewers an insight into the generation process.

In [99]:
from bs4 import BeautifulSoup
import urllib
import random
import json, ast
import nltk
import time
import re
import pronouncing
from nltk import tokenize
from nltk import pos_tag
from rhyme import rhymes_with
from nltk.corpus import cmudict
from nltk import corpus
from nltk import FreqDist
from time import sleep

# Preamble
Some text for while the user waits

In [430]:
print """These rhyming couplets are not always good.
But in the end, I did the best that I could.

Each day, I read The Dartmouth news.
And finds some sentences that I can use.

I analyze grammar and rhyming, too.
And generates phrases all brand-new.

They call me the Couplet Generating Robot.
The first sentences are real, the seconds are not.

Right now I am training, but soon you will see.
All of the rhymes that I have for thee..."""

These rhyming couplets are not always good.
But in the end, I did the best that I could.

Each day, I read The Dartmouth news.
And finds some sentences that I can use.

I analyze grammar and rhyming, too.
And generates phrases all brand-new.

They call me the Couplet Generating Robot.
The first sentences are real, the seconds are not.

Right now I am training, but soon you will see.
All of the rhymes that I have for thee...


# Headline Scraper

In [100]:
def headline_scrape(section):
    url = "http://www.thedartmouth.com/section/" + section
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,"html.parser")
    headlines = [j for j in soup.findAll("h4", {"class":"art-left-headline no-marg"})]

    for headline in headlines:
        for a in headline.find_all('a', href=True):
            unistring = a['href']
            front_page.append(unistring.encode("utf-8"))

            title_unistring = a.text
            page_name.append(title_unistring.encode("utf-8"))

In [101]:
front_page = []
page_name = []

sections = ["news", "arts", "sports", "opinion", "mirror"]

for section in sections:
    headline_scrape(section)

# Article Scraper

In [102]:
# uses the urls to get the article texts for each headline and stores them in a dictionary
article_dict = {}

length = len(front_page)

for i in range(0, length):
    url = front_page[i]
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,"html.parser")

    article = soup.find("div", {"class":"col-md-8 article-pg"})
    paras = article.find_all("p", {"class":None})

    article_text = ""
    for para in paras:
        article_text += para.text + " "

    article_text = article_text.strip()
    article_text = re.sub(u"\".*?\" ", "", article_text)
    
    article_dict[page_name[i]] = article_text.lower()

# Word Syllable Counter
https://stackoverflow.com/questions/5876040/number-of-syllables-for-words-in-a-text

In [103]:
d = cmudict.dict()
def nsyl(word):
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
    else:
        return [0]

# Bigram Dictionary

In [258]:
from nltk import ngrams

bigram_dict = {}
for j in range(0, length):
    article = article_dict[page_name[j]]
    article_clean = re.sub(r'([^\s\w]|_)+', "", article)
    
    article_words = article_clean.split()
    
    article_bigrams = list(ngrams(article_words, 2))

    for i in article_bigrams:
        if i[0] not in bigram_dict:
            bigram_dict[i[0]] = []
        bigram_dict[i[0]].append(i[1])

In [259]:
for key in bigram_dict:
    if bigram_dict[key] is None:
        del bigram_dict[key]

# Sentence Syllable Counter

In [106]:
def sent_syll(sent):
    words = sent.split()
    
    syll = 0
    
    for word in words:
        w = re.sub(r'([^\s\w]|_)+', "", word)
        s = nsyl(w)[0]
        if s != 0:
            syll += s
        else:
            return None
        
    return syll

The code below gets all sentences under 15 syllables and stores them in a list. 15 was chosen arbitrarily because I thought it would be easier from a coherence and run-time efficiency standpoint to work with shorter sentences.

In [429]:
all_articles = ""
for a in article_dict:
    all_articles += article_dict[a]
    
art_sents = tokenize.sent_tokenize(all_articles)

shorts = []

for i in art_sents:
    if sent_syll(i) <= 15 and sent_syll(i) is not None:
        shorts.append(i)

In [130]:
def sent_build():
    sent = random.choice(shorts)
    sent_clean = re.sub(r'([^\s\w]|_)+', "", sent)
    
    tag = pos_tag(tokenize.word_tokenize(sent_clean))
    
    gram = []
    for i in tag:
        gram.append(i[1])
        
    print sent
    return gram
    

# Part of Speech Dictionary

In [239]:
pos_dict = {}
for k in range(0, length):
    article = article_dict[page_name[k]]
    
    article_clean = re.sub(u"(\u2018|\u2019|\u201c)", "", article)
    
    tokenized_article = tokenize.word_tokenize(article_clean)
    
    pos_tags = pos_tag(tokenized_article)
    
    for p in pos_tags:
        if p[1] not in pos_dict:
            pos_dict[p[1]] = []
        if len(p[0]) > 1 and '.' not in p[0]:
            pos_dict[p[1]].append(p[0])

# Word to POS Dictionary

In [421]:
word_to_pos = {}
for k in range(0, length):
    article = article_dict[page_name[k]]
    
    article_clean = re.sub(u"(\u2018|\u2019|\u201c)", "", article)
    
    tokenized_article = tokenize.word_tokenize(article_clean)
    
    pos_tags = pos_tag(tokenized_article)
    
    for p in pos_tags:
        if p[0] not in word_to_pos:
            word_to_pos[p[0]] = []
        if len(p[1]) > 1 and '.' not in p[1]:
            word_to_pos[p[0]].append(p[1])
            
for w in word_to_pos:
    if word_to_pos[w] is None:
        del word_to_pos[w]
    else:
        word_to_pos[w] = list(set(word_to_pos[w]))

In [423]:
#word_to_pos["run"]

['VB', 'VBP', 'VBN', 'NN']

# Couplet Generation

This function returns a sentence and its part of speech tagging.

In [194]:
def sent_gram(num):
    sent = shorts[num]
    sent_clean = re.sub(r'([^\s\w]|_)+', "", sent)
    
    tag = pos_tag(tokenize.word_tokenize(sent_clean))
    
    gram = []
    for i in tag:
        gram.append(i[1])
        
    return (sent, gram)

This gets the frequency distribution of all the words in the Brown corpora for help with rhyming.

In [176]:
brown = FreqDist(corpus.brown.words())

4

This generates a couplet. The first line is a sentence chosen from the shorts list (list with all the shortish sentences from the day's articles). The second line is a randomly generated line based on the grammar and syllable length of the first sentence and then a rhyming word at the end of the sentence.

In [396]:
def couplet(num):
    sent = sent_gram(num)[0]
    gram = sent_gram(num)[1]

    words = re.sub(r'([^\s\w]|_)+', "", sent).split()
    rhymes = rhymes_with(words[-1])
    rhyme = ""
    for r in rhymes:
        if brown[r] > 0 and words[-1] not in r:
            rhyme = r
            
    if rhyme == "":
        return None
            
    new_sent = ""
    tries = 0
    while sent_syll(new_sent) != sent_syll(sent) and tries <= 100:
        new_sent = ""
        for s in gram[0:-1]:
            new_sent += random.choice(pos_dict[s]) + " "
        new_sent += rhyme
        tries += 1
    
    print sent
    print new_sent + sent[-1]

In [400]:
#couplet(0)

it is not, however, how it is.
him is there especially how she whiz.


This function will sometimes come out pretty bizarre but it tries to build a couplet using rhyming, part of speech tagging, and bigrams!

In [412]:
def second_line(sentence):
    sent_clean = re.sub(r'([^\s\w]|_)+', "", sentence)
    tag = pos_tag(tokenize.word_tokenize(sent_clean))[0:-1]
    
    gram = []
    for i in tag:
        gram.append(i[1])
    
    prev = random.choice(pos_dict[gram[0]])
    line = prev + " "
    
    for g in gram[1:]:
        if prev in bigram_dict:
            for word in bigram_dict[prev]:
                if word in word_to_pos and g in word_to_pos[word]:
                    line += word + " "
                    break
        else:
            random.choice(pos_dict[g])
    
    rhyme_word = sent_clean.split()[-1]
    rhymes = rhymes_with(rhyme_word)
    
    rhyme = ""
    for r in rhymes:
        if brown[r] > 0:
            rhyme = r
            
    if rhyme == "":
        return None
    
    print sentence
    print line + rhyme + "."
    

This code compiles all of the article into one string and then breaks them up by sentences.

In [314]:
all_articles = ""
for a in article_dict:
    all_articles += article_dict[a]
    
art_sents = tokenize.sent_tokenize(all_articles)

This couplet takes in a sentence from the shorts list and then finds a rhyming word somewhere in any of the articles and builds the second line by the word's position. The second line is the same amount of words as the first line (except when its word placement in the sentence is earlier than the length of the first sentence).

In [408]:
def find_rhyme(sentence):
    cleaner = re.sub(r'([^\s\w]|_)+', "", sentence)
    
    word_sent = cleaner.split()
    sent_length = len(word_sent)
    
    rhymes = rhymes_with(word_sent[-1])

    rhyme_sent = ""
    where = 10000
    for sent in art_sents:
        for r in rhymes:
            finder = " " + r + " "
            val = sent.find(finder)
            
            if finder in sent and val < where:
                s = sent.split()
                ind = s.index(r)
                dist = 0
                if ind - sent_length > 0:
                    dist = ind - sent_length + 1
                rhyme_sent = s[dist:ind+1]
                
    if rhyme_sent == "":
        return None

    print sentence
    print " ".join(rhyme_sent) + sentence[-1]

In [409]:
#find_rhyme(shorts[0])

it is not, however, how it is.
a foul and made both of his.


# Putting it all together!
Here is an infinite loop that continuously prints out couplets using three styles of generation. Some come out well, some don't. But rhyming is fun, so enjoy.

In [426]:
while True:
    couplet(random.randint(0, len(shorts)-1))
    sleep(3)
    print "\n"
    find_rhyme(random.choice(shorts))
    sleep(3)
    print "\n"
    second_line(random.choice(shorts))
    sleep(3)
    print "\n"

after three missed calls, my father finally dials me back.
of two discovered beliefs its history so is he wrack.


oprah intended to empower marginalized voices.
multiple critical innovations and bold choices.


we also had our first few upsets of the season!
it i was your i i helps because a treason.




flesch wields her power well.
synagogues in pittsburgh, i tell.


my chest burns with the fire of terror.
their finest properties service error.




what is the line and how strict should it be?
on oct. 27. your legacy will live on in the?


never again can we pray without fear.
as early we i though year.




i’m vegan, but i’m not allergic to any foods.
this piece of sports memorabilia also includes.


we all just feed off each other really well.
we all never spent all spent never yell.




it can be a very fine line.
could occur in a community like mine.


through understanding comes compassion.
in voting shanges ration.


what do you hope your students take away from your class?
wh



as for the future, ortiz is uncertain but excited.
of life congregation forever altered synagogues in the united.




they are two different things.
us are 2018 dominant wrings.


in trying times, it is easy to place blame.
people across the country who now know your name.




desperately, i ask him if our family is safe.
easily discordant love themselves with his laurie thinks strafe.


we have time to correct it and we’ll watch video on it.
control over her daughters, adora is most fond of those who submit.


this is my opinion, which is based on the truth.
the dartmouth training dartmouth proposed past no youth.


gun violence in america is an epidemic.
presence level of psychology is neither systemic.






to my relative rose: i wish i knew you better.
to your hate were top open trent gave myself sweater.




the desire to be right is the thirst for truth.
the training open new dartmouth no training past youth.


we also had our first few upsets of the season!
you only pulled hi

KeyboardInterrupt: 